
---

# 🧾 FastAPI CRUD API with SQLite and Pydantic v2

This example demonstrates a **full CRUD API** using **FastAPI**, **SQLite**, and **Pydantic v2** models for validation and serialization.

---

## ✅ Features Covered

| Feature               | Description                                 |
| --------------------- | ------------------------------------------- |
| SQLite DB Integration | Lightweight embedded DB for CRUD operations |
| Pydantic v2 Models    | Input validation & response serialization   |
| Response Models       | Clean output using `response_model`         |
| HTTPException         | Proper 404 error handling                   |

---

## 📦 Database & App Setup

```python
from contextlib import asynccontextmanager
import sqlite3
from typing import List

from fastapi import FastAPI, HTTPException, status
from pydantic import BaseModel, Field

APP_TITLE = "FastAPI + SQLite + Pydantic v2"
DB_PATH = "employee.db"

def get_db() -> sqlite3.Connection:
    conn = sqlite3.connect(DB_PATH, check_same_thread=False)
    conn.row_factory = sqlite3.Row
    return conn

def init_db() -> None:
    with get_db() as db:
        db.execute("""
            CREATE TABLE IF NOT EXISTS employee (
                id   INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT    NOT NULL,
                age  INTEGER NOT NULL,
                city TEXT    NOT NULL
            )
        """)
        db.commit()

@asynccontextmanager
async def lifespan(_: FastAPI):
    init_db()   # ensure table exists at startup
    yield

app = FastAPI(title=APP_TITLE, version="1.0.0", lifespan=lifespan)
```

---

## 🧾 Pydantic Models

```python
class EmployeeCreate(BaseModel):
    name: str = Field(..., min_length=1, examples=["John Doe"])
    age: int = Field(..., ge=0, examples=[30])
    city: str = Field(..., min_length=1, examples=["New York"])

class EmployeeUpdate(BaseModel):
    name: str = Field(..., min_length=1, examples=["Updated Name"])
    age: int = Field(..., ge=0, examples=[35])
    city: str = Field(..., min_length=1, examples=["Los Angeles"])

class EmployeeOut(BaseModel):
    id: int
    name: str
    age: int
    city: str

class Message(BaseModel):
    message: str
```

---

## 🚀 API Endpoints

### 🔹 Home Page

```python
@app.get("/", response_model=Message, summary="Landing / health")
def home_page():
    return {"message": "Welcome to landing page of FastAPI"}
```

---

### 🔹 Create Employee

```python
@app.post("/create", response_model=EmployeeOut, status_code=status.HTTP_201_CREATED, summary="Create employee")
def insert_record(employee: EmployeeCreate):
    with get_db() as db:
        cur = db.execute(
            "INSERT INTO employee (name, age, city) VALUES (?, ?, ?)",
            (employee.name, employee.age, employee.city)
        )
        db.commit()
        new_id = cur.lastrowid
        row = db.execute("SELECT id, name, age, city FROM employee WHERE id = ?", (new_id,)).fetchone()
        return EmployeeOut.model_validate(dict(row))
```

---

### 🔹 View All Employees

```python
@app.get("/employee/view", response_model=List[EmployeeOut], summary="List all employees")
def view_records():
    with get_db() as db:
        cursor = db.execute("SELECT id, name, age, city FROM employee ORDER BY id ASC")
        employees = cursor.fetchall()
    return [EmployeeOut.model_validate(dict(emp)) for emp in employees]
```

---

### 🔹 View Single Employee

```python
@app.get("/employee/view/{employee_id}", response_model=EmployeeOut, summary="Get one employee")
def single_view_record(employee_id: int):
    with get_db() as db:
        employee = db.execute(
            "SELECT id, name, age, city FROM employee WHERE id = ?",
            (employee_id,)
        ).fetchone()
        if employee is None:
            raise HTTPException(status_code=404, detail="Employee not found")
    return EmployeeOut.model_validate(dict(employee))
```

---

### 🔹 Update Employee

```python
@app.put("/employee/update/{employee_id}", response_model=EmployeeOut, summary="Full update employee")
def single_update_record(employee_id: int, employee: EmployeeUpdate):
    with get_db() as db:
        exists = db.execute("SELECT 1 FROM employee WHERE id = ?", (employee_id,)).fetchone()
        if not exists:
            raise HTTPException(status_code=404, detail="Employee not found")
        db.execute(
            "UPDATE employee SET name = ?, age = ?, city = ? WHERE id = ?",
            (employee.name, employee.age, employee.city, employee_id)
        )
        db.commit()
        row = db.execute("SELECT id, name, age, city FROM employee WHERE id = ?", (employee_id,)).fetchone()
        return EmployeeOut.model_validate(dict(row))
```

---

### 🔹 Delete Employee

```python
@app.delete("/employee/delete/{employee_id}", response_model=Message, summary="Delete employee")
def single_record_delete(employee_id: int):
    with get_db() as db:
        exists = db.execute("SELECT 1 FROM employee WHERE id = ?", (employee_id,)).fetchone()
        if not exists:
            raise HTTPException(status_code=404, detail="Employee not found")
        db.execute("DELETE FROM employee WHERE id = ?", (employee_id,))
        db.commit()
    return {"message": f"Employee id {employee_id} deleted successfully"}
```

---

## 🧪 To Run the App

```bash
uvicorn crud_pydantic:app --reload
```

> Replace `crud_pydantic` with your actual filename if different.

---
